# ☁️ GCP para Ingeniería de Datos: BigQuery, Cloud Storage, Dataflow y Composer

Este notebook introduce el ecosistema de Google Cloud Platform (GCP) para Data Engineering, cubriendo almacenamiento en Cloud Storage, transformaciones con BigQuery y Dataflow, y orquestación con Cloud Composer.

**Autor:** LuisRai (Luis J. Raigoso V.)  
**Nivel:** Mid  
**Duración:** 90-120 minutos

## ⚠️ RECORDATORIO IMPORTANTE

### 🚨 NOTEBOOKS vs PRODUCCIÓN

Este curso usa notebooks para **enseñanza**, pero en tu trabajo real:

**❌ NO uses notebooks para pipelines en producción**

**✅ USA:**
- Scripts Python modulares en `src/`
- Cloud Composer DAGs (Airflow managed)
- Cloud Functions para event-driven processing
- Cloud Build para CI/CD
- Terraform para Infrastructure as Code

---

**Autor:** LuisRai (Luis J. Raigoso V.) | © 2024-2025

---

## Requisitos y Notas de Ejecución

- Para ejecutar código real de GCP necesitas:
  - Proyecto GCP creado
  - Credenciales configuradas (`gcloud auth login` o service account JSON)
  - Billing habilitado
  - APIs habilitadas: BigQuery, Cloud Storage, Dataflow, Composer
- **Nunca subas credenciales al repositorio**
- Usa variables de entorno o Google Cloud SDK
- Este notebook muestra ejemplos ejecutables si tienes un proyecto GCP configurado

### ☁️ **Google Cloud Platform: Ecosistema para Data Engineering**

**Stack Moderno de GCP para Datos:**

1. **Cloud Storage (GCS)**: Object storage para Data Lakes
   - Similar a S3, pero con modelo de consistencia fuerte desde el inicio
   - Clases: Standard, Nearline (30d), Coldline (90d), Archive (365d)
   - Pricing: ~$0.020/GB/mes (Standard en multi-region)

2. **BigQuery**: Data Warehouse serverless con SQL
   - Almacenamiento columnar comprimido
   - Separación compute/storage (paga solo por queries ejecutadas)
   - Pricing: $5/TB escaneado (on-demand) o flat-rate mensual
   - Streaming inserts: $0.01 per 200MB

3. **Dataflow**: Procesamiento stream/batch con Apache Beam
   - Serverless, auto-scaling
   - Unified model: mismo código para batch y streaming
   - Pricing: por vCPU-hora + GB-hora (workers)

4. **Cloud Composer**: Airflow totalmente administrado
   - DAGs en Python, GKE-based
   - Integración nativa con servicios GCP
   - Pricing: por tamaño de environment + compute

5. **Cloud Functions**: Funciones serverless event-driven
   - Triggers: HTTP, Cloud Storage, Pub/Sub, Firestore
   - Runtime: Python 3.7-3.11, Node.js, Go, Java
   - Pricing: por invocaciones + compute time

**Arquitectura de Referencia:**
```
Fuentes → [Pub/Sub] → Cloud Storage (raw) → [Dataflow/Cloud Functions] 
                           ↓
                    Cloud Storage (curated) → BigQuery (analítica)
                           ↓
                    Cloud Composer (orquestación)
```

**Ventajas de GCP para Datos:**
- **BigQuery**: Queries extremadamente rápidas (MPP distribuido)
- **Integración ML**: BigQuery ML, Vertex AI
- **Consistencia fuerte**: No eventual consistency issues
- **Kubernetes nativo**: GKE para workloads custom

**Comparación con AWS:**

| Servicio | GCP | AWS |
|----------|-----|-----|
| Object Storage | Cloud Storage | S3 |
| Data Warehouse | BigQuery | Redshift |
| ETL Serverless | Dataflow (Beam) | Glue (PySpark) |
| Streaming | Dataflow + Pub/Sub | Kinesis + Lambda |
| Orchestration | Cloud Composer | MWAA (Airflow) |
| Functions | Cloud Functions | Lambda |

---
**Autor:** Luis J. Raigoso V. (LJRV)

## 1. Cloud Storage: Data Lake Básico

### 🗄️ **Cloud Storage: Fundamentos**

**Conceptos Core:**

- **Bucket**: Contenedor global único
  - Nombre: `mi-data-lake-gcp` (minúsculas, números, guiones)
  - Location: `us-central1`, `europe-west1`, `us` (multi-region)
  - Storage class: Standard, Nearline, Coldline, Archive

- **Object**: Archivo con metadata
  - Key: `gs://bucket/path/to/file.csv`
  - Metadata: Content-Type, custom headers
  - Versionamiento: Object Versioning (similar a S3)

**Organizacion Recomendada:**

```
gs://my-datalake/
├── raw/                    ← Datos crudos
│   ├── sales/
│   │   └── 2025/10/30/
│   └── customers/
├── staging/                ← Datos en proceso
└── curated/                ← Datos procesados
    ├── sales_aggregated/
    └── customer_metrics/
```

**Lifecycle Management:**
```json
{
  "lifecycle": {
    "rule": [
      {
        "action": {"type": "SetStorageClass", "storageClass": "NEARLINE"},
        "condition": {"age": 30}
      },
      {
        "action": {"type": "Delete"},
        "condition": {"age": 365}
      }
    ]
  }
}
```

**Operaciones con Python Client:**
```python
from google.cloud import storage

client = storage.Client()

# Crear bucket
bucket = client.create_bucket('my-bucket', location='us-central1')

# Subir archivo
blob = bucket.blob('raw/ventas.csv')
blob.upload_from_filename('ventas.csv')

# Listar objetos
for blob in bucket.list_blobs(prefix='raw/'):
    print(blob.name)

# Descargar archivo
blob.download_to_filename('downloaded.csv')
```

**Costos:**
- Storage: $0.020/GB/mes (Standard)
- Class A operations (write): $0.05 per 10,000 ops
- Class B operations (read): $0.004 per 10,000 ops
- Network egress: $0.12/GB (fuera de GCP)

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [ ]:
# Configuración (sin ejecutar sin proyecto GCP real)
PROJECT_ID = 'mi-proyecto-gcp'
BUCKET_NAME = f'{PROJECT_ID}-datalake'
LOCATION = 'us-central1'

print(f'📋 Configuración: {PROJECT_ID} / {BUCKET_NAME}')
print('⚠️ Requiere: gcloud auth login o service account JSON')

### 1.1 Crear bucket y subir datos

In [ ]:
# Ejemplo de código (descomentar con proyecto real)
'''
from google.cloud import storage
import pandas as pd

# Inicializar cliente
client = storage.Client(project=PROJECT_ID)

# Crear bucket
try:
    bucket = client.create_bucket(BUCKET_NAME, location=LOCATION)
    print(f'✅ Bucket {BUCKET_NAME} creado')
except Exception as e:
    print(f'ℹ️ Bucket ya existe o error: {e}')
    bucket = client.bucket(BUCKET_NAME)

# Subir CSV
df = pd.read_csv('../../datasets/raw/ventas.csv')
csv_string = df.to_csv(index=False)

blob = bucket.blob('raw/ventas/ventas_2025_10.csv')
blob.upload_from_string(csv_string, content_type='text/csv')
print('📤 Archivo subido a Cloud Storage')
'''
print('Código de ejemplo listo para ejecutar con proyecto GCP')

## 2. BigQuery: Data Warehouse Serverless

### 📊 **BigQuery: SQL Analytics a Escala**

**Arquitectura:**

BigQuery separa **storage** y **compute**:
- Storage: Columnar format (Capacitor), compresión automática
- Compute: Dremel engine (MPP distribuido con miles de workers)

**Beneficios:**
- Queries sobre TB/PB en segundos
- No administrar clusters (100% serverless)
- Standard SQL (ANSI SQL 2011 compatible)
- Integración con herramientas BI (Looker, Tableau, Data Studio)

**Conceptos:**

1. **Dataset**: Contenedor lógico de tablas
   - Similar a "database" en SQL tradicional
   - Permisos a nivel dataset

2. **Table**: Datos estructurados
   - Native tables: Datos en BigQuery storage
   - External tables: Data en GCS (federated queries)
   - Partitioned tables: Por fecha/rango (reduce scan)
   - Clustered tables: Por columnas específicas (mejor performance)

3. **View**: Query guardada
   - Authorized views: Control de acceso granular

**Particionamiento:**
```sql
-- Tabla particionada por fecha
CREATE TABLE `project.dataset.sales_partitioned`
PARTITION BY DATE(order_date)
AS SELECT * FROM `project.dataset.sales_raw`;

-- Query optimizada (scan solo 1 día)
SELECT * FROM `project.dataset.sales_partitioned`
WHERE order_date = '2025-10-30';
```

**Clustering:**
```sql
-- Tabla clusterizada por customer_id, product_id
CREATE TABLE `project.dataset.sales_clustered`
PARTITION BY DATE(order_date)
CLUSTER BY customer_id, product_id
AS SELECT * FROM `project.dataset.sales_raw`;

-- Query beneficiada (pre-sorted data)
SELECT * FROM `project.dataset.sales_clustered`
WHERE customer_id = 12345;
```

**BigQuery ML (Machine Learning integrado):**
```sql
-- Crear modelo de regresión logística
CREATE OR REPLACE MODEL `project.dataset.churn_model`
OPTIONS(model_type='logistic_reg') AS
SELECT
  customer_id,
  age,
  total_purchases,
  churned as label
FROM `project.dataset.customers`;

-- Predecir
SELECT * FROM ML.PREDICT(
  MODEL `project.dataset.churn_model`,
  (SELECT * FROM `project.dataset.new_customers`)
);
```

**Best Practices:**
- Particionar tablas grandes (>1GB)
- Evitar `SELECT *` (especifica columnas)
- Usar `_TABLE_SUFFIX` para tablas wildcard
- Aprovechar result caching (24h gratuito)
- Usar slots reservation para cargas predecibles

**Pricing:**
- On-demand: $5/TB escaneado
- Flat-rate: desde $2,000/mes (100 slots)
- Storage: $0.020/GB active, $0.010/GB long-term (90d+)

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [ ]:
# Configuración BigQuery
DATASET_ID = 'data_engineering_course'
TABLE_ID = 'ventas'

print(f'📊 Dataset: {PROJECT_ID}.{DATASET_ID}')
print(f'📋 Tabla: {TABLE_ID}')

### 2.1 Crear dataset y tabla desde Cloud Storage

In [ ]:
# Ejemplo BigQuery
'''
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID)

# Crear dataset
dataset_ref = client.dataset(DATASET_ID)
try:
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = LOCATION
    dataset = client.create_dataset(dataset)
    print(f'✅ Dataset {DATASET_ID} creado')
except Exception as e:
    print(f'ℹ️ Dataset ya existe: {e}')

# Cargar CSV desde GCS a BigQuery
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField('venta_id', 'INTEGER'),
        bigquery.SchemaField('cliente_id', 'INTEGER'),
        bigquery.SchemaField('producto_id', 'INTEGER'),
        bigquery.SchemaField('cantidad', 'INTEGER'),
        bigquery.SchemaField('total', 'FLOAT'),
        bigquery.SchemaField('fecha', 'DATE'),
    ],
    skip_leading_rows=1,
    source_format=bigquery.SourceFormat.CSV,
    write_disposition='WRITE_TRUNCATE',
)

uri = f'gs://{BUCKET_NAME}/raw/ventas/ventas_2025_10.csv'
table_ref = dataset_ref.table(TABLE_ID)

load_job = client.load_table_from_uri(uri, table_ref, job_config=job_config)
load_job.result()  # Wait for job to complete

print(f'✅ Cargados {load_job.output_rows} registros en {TABLE_ID}')
'''
print('Código BigQuery listo para ejecutar')

### 2.2 Queries SQL en BigQuery

In [ ]:
# Ejemplo de queries
'''
# Query simple
query = f"""
SELECT 
  cliente_id,
  SUM(total) as total_ventas,
  COUNT(*) as num_ventas
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
GROUP BY cliente_id
ORDER BY total_ventas DESC
LIMIT 10
"""

query_job = client.query(query)
results = query_job.result()

for row in results:
    print(f'Cliente {row.cliente_id}: ${row.total_ventas:.2f} ({row.num_ventas} ventas)')

# Metadata del job
print(f'\\n📊 Query Stats:')
print(f'  Bytes processed: {query_job.total_bytes_processed / 1e9:.2f} GB')
print(f'  Bytes billed: {query_job.total_bytes_billed / 1e9:.2f} GB')
print(f'  Cost estimate: ${(query_job.total_bytes_billed / 1e12) * 5:.4f}')
'''
print('Queries BigQuery con estimación de costos')

## 3. Dataflow: Procesamiento con Apache Beam

### 🌊 **Dataflow: Unified Stream/Batch Processing**

**Apache Beam Concepts:**

Beam es un modelo de programación unificado para batch y streaming:

```
Pipeline → PCollection → Transform → PCollection → ...
```

- **Pipeline**: Grafo de transformaciones
- **PCollection**: Conjunto distribuido de datos (immutable)
- **Transform**: Operación sobre PCollection (Map, Filter, GroupByKey, etc.)

**Runners:**
- DirectRunner: Local (testing)
- DataflowRunner: GCP managed service
- FlinkRunner: Apache Flink
- SparkRunner: Apache Spark

**Patrón Básico:**
```python
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

options = PipelineOptions([
    '--project=my-project',
    '--region=us-central1',
    '--runner=DataflowRunner',
    '--temp_location=gs://my-bucket/temp',
])

with beam.Pipeline(options=options) as pipeline:
    (pipeline
     | 'Read' >> beam.io.ReadFromText('gs://input/*.csv')
     | 'Parse' >> beam.Map(lambda line: line.split(','))
     | 'Filter' >> beam.Filter(lambda row: float(row[2]) > 100)
     | 'Format' >> beam.Map(lambda row: f'{row[0]},{row[1]}')
     | 'Write' >> beam.io.WriteToText('gs://output/result'))
```

**Windowing (Streaming):**
```python
from apache_beam import window

(events
 | 'Window' >> beam.WindowInto(window.FixedWindows(60))  # 1-min windows
 | 'Sum' >> beam.CombinePerKey(sum)
 | 'Write' >> beam.io.WriteToBigQuery('project:dataset.table'))
```

**Transforms Comunes:**

1. **ParDo** (Parallel Do):
   ```python
   class ExtractFields(beam.DoFn):
       def process(self, element):
           parts = element.split(',')
           yield {'id': int(parts[0]), 'value': float(parts[1])}
   
   data | beam.ParDo(ExtractFields())
   ```

2. **GroupByKey**:
   ```python
   # (key, value) pairs → (key, [values])
   pairs | beam.GroupByKey()
   ```

3. **CombinePerKey**:
   ```python
   # Más eficiente que GroupByKey + Map
   pairs | beam.CombinePerKey(sum)
   ```

4. **Flatten**:
   ```python
   # Merge múltiples PCollections
   (pcoll1, pcoll2, pcoll3) | beam.Flatten()
   ```

**Side Inputs (Broadcasting):**
```python
lookup_table = (pipeline
                | 'Read Lookup' >> beam.io.ReadFromText('gs://lookup.csv')
                | 'Parse' >> beam.Map(lambda x: x.split(',')))

main_data | beam.Map(
    lambda x, table: enrich(x, table),
    beam.pvalue.AsDict(lookup_table)
)
```

**Dataflow vs Alternatives:**

| Aspecto | Dataflow | Spark | Flink |
|---------|----------|-------|-------|
| **Model** | Beam (unified) | RDD/DataFrame | DataStream |
| **Serverless** | ✅ Sí | ❌ No (EMR/Databricks) | ❌ No |
| **Exactly-once** | ✅ Sí | ⚠️ Difícil | ✅ Sí |
| **Late Data** | ✅ Excellent | ⚠️ Manual | ✅ Good |
| **Learning Curve** | Media | Baja | Alta |

**Use Cases:**
- ETL masivos (TB → PB)
- Real-time analytics (con Pub/Sub)
- ML feature engineering
- Data quality validation

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [ ]:
# Ejemplo de pipeline Dataflow
dataflow_example = '''
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

class ParseCSV(beam.DoFn):
    def process(self, element):
        parts = element.split(',')
        yield {
            'venta_id': int(parts[0]),
            'cliente_id': int(parts[1]),
            'total': float(parts[4])
        }

class FilterHighValue(beam.DoFn):
    def process(self, element):
        if element['total'] > 100:
            yield element

options = PipelineOptions([
    '--project=PROJECT_ID',
    '--region=us-central1',
    '--runner=DataflowRunner',
    '--temp_location=gs://BUCKET/temp',
    '--staging_location=gs://BUCKET/staging',
])

with beam.Pipeline(options=options) as pipeline:
    (pipeline
     | 'Read GCS' >> beam.io.ReadFromText('gs://BUCKET/raw/ventas/*.csv')
     | 'Skip Header' >> beam.Filter(lambda line: not line.startswith('venta_id'))
     | 'Parse' >> beam.ParDo(ParseCSV())
     | 'Filter' >> beam.ParDo(FilterHighValue())
     | 'Aggregate' >> beam.Map(lambda x: (x['cliente_id'], x['total']))
     | 'Group' >> beam.CombinePerKey(sum)
     | 'Format' >> beam.Map(lambda kv: f"{kv[0]},{kv[1]}")
     | 'Write' >> beam.io.WriteToText('gs://BUCKET/curated/ventas_summary'))
'''

print(dataflow_example)
print('\\n💡 Para ejecutar: python dataflow_job.py')

## 4. Cloud Composer: Airflow Administrado

### 🎼 **Cloud Composer: Airflow en GCP**

**¿Qué es Cloud Composer?**

Servicio totalmente administrado de Apache Airflow en GCP:
- Basado en GKE (Google Kubernetes Engine)
- Auto-scaling de workers
- Integración nativa con servicios GCP
- Monitoreo con Cloud Logging/Monitoring

**Componentes:**

1. **Environment**: Cluster Airflow dedicado
   - Web server (UI)
   - Scheduler
   - Workers (Celery o Kubernetes)
   - Database (Cloud SQL PostgreSQL)

2. **DAGs Folder**: GCS bucket automático
   - `gs://[bucket]/dags/`
   - Sync automático al subir DAGs

**Operators GCP:**

```python
from airflow.providers.google.cloud.operators.bigquery import (
    BigQueryCreateEmptyDatasetOperator,
    BigQueryInsertJobOperator,
)
from airflow.providers.google.cloud.operators.gcs import (
    GCSCreateBucketOperator,
    GCSDeleteBucketOperator,
)
from airflow.providers.google.cloud.transfers.gcs_to_bigquery import (
    GCSToBigQueryOperator,
)

with DAG('gcp_pipeline', schedule_interval='@daily') as dag:
    
    create_dataset = BigQueryCreateEmptyDatasetOperator(
        task_id='create_dataset',
        dataset_id='my_dataset',
        project_id=PROJECT_ID,
    )
    
    load_to_bq = GCSToBigQueryOperator(
        task_id='load_csv_to_bq',
        bucket='my-bucket',
        source_objects=['raw/ventas/*.csv'],
        destination_project_dataset_table=f'{PROJECT_ID}.my_dataset.ventas',
        schema_fields=[
            {'name': 'venta_id', 'type': 'INTEGER'},
            {'name': 'cliente_id', 'type': 'INTEGER'},
            {'name': 'total', 'type': 'FLOAT'},
        ],
        write_disposition='WRITE_TRUNCATE',
    )
    
    run_query = BigQueryInsertJobOperator(
        task_id='aggregate_sales',
        configuration={
            'query': {
                'query': '''
                    CREATE OR REPLACE TABLE `{}.my_dataset.sales_summary` AS
                    SELECT cliente_id, SUM(total) as total
                    FROM `{}.my_dataset.ventas`
                    GROUP BY cliente_id
                '''.format(PROJECT_ID, PROJECT_ID),
                'useLegacySql': False,
            }
        },
    )
    
    create_dataset >> load_to_bq >> run_query
```

**Dataflow Operator:**
```python
from airflow.providers.google.cloud.operators.dataflow import (
    DataflowCreatePythonJobOperator,
)

run_dataflow = DataflowCreatePythonJobOperator(
    task_id='dataflow_etl',
    py_file='gs://my-bucket/dataflow/pipeline.py',
    job_name='ventas-etl',
    options={
        'project': PROJECT_ID,
        'region': 'us-central1',
        'tempLocation': 'gs://my-bucket/temp',
    },
)
```

**Environment Variables & Connections:**

```python
from airflow.models import Variable

# Airflow Variables (en UI o CLI)
PROJECT_ID = Variable.get('gcp_project_id')
BUCKET = Variable.get('data_bucket')

# Connections (GCP → Airflow Connection)
# ID: google_cloud_default
# Type: Google Cloud
# Keyfile JSON: [service account JSON]
```

**Pricing:**
- Environment: ~$300-500/mes (pequeño)
- Compute: Workers + scheduler
- Storage: GCS para DAGs + logs
- Database: Cloud SQL (managed)

**Alternatives:**
- Managed Airflow (MWAA en AWS)
- Self-hosted Airflow (Kubernetes)
- Cloud Workflows (simple orchestration)

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [ ]:
# Ejemplo de DAG para Composer
composer_dag = '''
from airflow import DAG
from airflow.providers.google.cloud.transfers.gcs_to_bigquery import GCSToBigQueryOperator
from airflow.providers.google.cloud.operators.bigquery import BigQueryInsertJobOperator
from datetime import datetime, timedelta

default_args = {
    'owner': 'data-team',
    'depends_on_past': False,
    'start_date': datetime(2025, 1, 1),
    'email_on_failure': True,
    'email_on_retry': False,
    'retries': 3,
    'retry_delay': timedelta(minutes=5),
}

with DAG(
    'ventas_daily_etl',
    default_args=default_args,
    description='Pipeline diario de ventas',
    schedule_interval='0 2 * * *',  # 2 AM daily
    catchup=False,
    tags=['ventas', 'etl', 'gcp'],
) as dag:
    
    load_raw = GCSToBigQueryOperator(
        task_id='load_raw_data',
        bucket='my-datalake',
        source_objects=['raw/ventas/{{ ds }}/*.csv'],
        destination_project_dataset_table='project.dataset.ventas_raw',
        write_disposition='WRITE_APPEND',
        skip_leading_rows=1,
    )
    
    transform = BigQueryInsertJobOperator(
        task_id='transform_data',
        configuration={
            'query': {
                'query': """
                    CREATE OR REPLACE TABLE `project.dataset.ventas_clean` AS
                    SELECT 
                        venta_id,
                        cliente_id,
                        CAST(total AS FLOAT64) as total,
                        DATE(fecha) as fecha
                    FROM `project.dataset.ventas_raw`
                    WHERE fecha = '{{ ds }}'
                      AND total > 0
                """,
                'useLegacySql': False,
            }
        },
    )
    
    load_raw >> transform

# Subir a: gs://[composer-bucket]/dags/ventas_etl.py
'''

print(composer_dag)
print('\\n💡 Subir a Cloud Composer DAGs folder en GCS')

## 5. Cloud Run: Containerized Serverless Workloads

### ⚡ **Cloud Functions: Serverless para Datos**

**Triggers Disponibles:**

1. **HTTP**: API endpoints
2. **Cloud Storage**: Object created/deleted/updated
3. **Pub/Sub**: Message queue events
4. **Firestore**: Document changes
5. **Cloud Scheduler**: Cron jobs

**Ejemplo: Procesar CSV al subir a GCS**

```python
# main.py
from google.cloud import bigquery
import pandas as pd

def process_csv(event, context):
    """
    Triggered by Cloud Storage when CSV uploaded.
    
    Args:
        event (dict): Event payload (file metadata)
        context (google.cloud.functions.Context): Event context
    """
    file_name = event['name']
    bucket_name = event['bucket']
    
    print(f'Processing file: gs://{bucket_name}/{file_name}')
    
    # Skip if not in raw/ prefix
    if not file_name.startswith('raw/'):
        print('Skipping non-raw file')
        return
    
    # Read CSV from GCS
    gcs_uri = f'gs://{bucket_name}/{file_name}'
    df = pd.read_csv(gcs_uri)
    
    # Transform
    df_clean = df.dropna()
    df_clean['total'] = df_clean['total'].astype(float)
    
    # Load to BigQuery
    client = bigquery.Client()
    table_id = 'project.dataset.ventas_processed'
    
    job_config = bigquery.LoadJobConfig(
        write_disposition='WRITE_APPEND',
    )
    
    job = client.load_table_from_dataframe(df_clean, table_id, job_config=job_config)
    job.result()
    
    print(f'Loaded {len(df_clean)} rows to {table_id}')
```

**requirements.txt:**
```
google-cloud-bigquery
google-cloud-storage
pandas
```

**Deploy:**
```bash
gcloud functions deploy process_csv \\
  --runtime python310 \\
  --trigger-resource my-bucket \\
  --trigger-event google.storage.object.finalize \\
  --entry-point process_csv \\
  --region us-central1 \\
  --memory 512MB \\
  --timeout 300s
```

**Best Practices:**
- Idempotent functions (puede ejecutarse múltiples veces)
- Timeout < 9 min (max 540s)
- Lightweight dependencies (cold start impact)
- Use Pub/Sub para retry logic
- Cloud Run para workloads >9 min

**Pub/Sub + Cloud Functions:**
```python
import base64
import json

def process_message(event, context):
    """Triggered from Pub/Sub topic"""
    pubsub_message = base64.b64decode(event['data']).decode('utf-8')
    data = json.loads(pubsub_message)
    
    print(f'Processing message: {data}')
    # ETL logic here
```

**Publish to Pub/Sub:**
```bash
gcloud pubsub topics publish data-events \\
  --message '{"file": "gs://bucket/data.csv", "type": "sales"}'
```

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [ ]:
# Ejemplo de Cloud Function completo
cloud_function_example = '''
# main.py - Cloud Function para validar y cargar datos
from google.cloud import bigquery, storage
import pandas as pd
import logging

def validate_and_load(event, context):
    """
    Cloud Function triggered por Cloud Storage.
    Valida CSV y carga a BigQuery.
    """
    file_name = event['name']
    bucket_name = event['bucket']
    
    logging.info(f'File: gs://{bucket_name}/{file_name}')
    
    # Read from GCS
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    content = blob.download_as_text()
    
    # Parse CSV
    from io import StringIO
    df = pd.read_csv(StringIO(content))
    
    # Validations
    required_cols = ['venta_id', 'cliente_id', 'total']
    if not all(col in df.columns for col in required_cols):
        raise ValueError(f'Missing columns: {required_cols}')
    
    if df['total'].isnull().sum() > 0:
        raise ValueError('Null values found in total column')
    
    # Clean
    df['total'] = df['total'].astype(float)
    df = df[df['total'] > 0]
    
    # Load to BigQuery
    bq_client = bigquery.Client()
    table_id = 'project.dataset.ventas'
    
    job = bq_client.load_table_from_dataframe(
        df, 
        table_id,
        job_config=bigquery.LoadJobConfig(
            write_disposition='WRITE_APPEND'
        )
    )
    job.result()
    
    logging.info(f'✅ Loaded {len(df)} rows')
    return f'Success: {len(df)} rows'

# Deploy:
# gcloud functions deploy validate_and_load \\
#   --runtime python310 \\
#   --trigger-resource my-bucket \\
#   --trigger-event google.storage.object.finalize \\
#   --entry-point validate_and_load
'''

print(cloud_function_example)

## 5. Cloud Run: Containerized Workloads Serverless

### 🐳 **Cloud Run: Contenedores Serverless para Cargas Personalizadas**

**¿Qué es Cloud Run?**

Cloud Run es una plataforma serverless completamente administrada que ejecuta contenedores Docker sin límite de 9 minutos (diferencia clave vs Cloud Functions).

**Características Principales:**

- **Basado en Kubernetes**: Compatible con estándar Knative
- **Sin límite de tiempo**: Soporta procesos largos (hasta horas)
- **Request-based scaling**: Escala de 0 a N instancias según tráfico
- **Portable**: Código se puede mover entre Cloud Run, GKE, on-prem
- **Pricing**: Paga por CPU/memoria durante tiempo de ejecución + requests

**Cuándo usar Cloud Run vs Cloud Functions:**

| Aspecto | Cloud Run | Cloud Functions |
|---------|-----------|-----------------|
| **Tiempo límite** | Sin límite práctico | 9 min max |
| **Lenguajes** | Cualquiera (Docker) | Python, Node, Go, Java |
| **Dependencias** | Control total | Limitado por entorno |
| **Custom binaries** | ✅ Sí | ❌ No |
| **Cold start** | ~1-3s | ~0.5-1s |
| **HTTP/gRPC** | ✅ Ambos | Solo HTTP |
| **Use case** | ETL largo, APIs, ML inference | Event handling, triggers rápidos |

**Arquitectura para Data Engineering:**

```
1. ETL LARGO (>9 min)
   Cloud Scheduler → Cloud Run → BigQuery
                       ↓
                  Cloud Storage (resultados)

2. API DE DATOS
   Load Balancer → Cloud Run Service
                       ↓
                  BigQuery / Cloud SQL

3. ML INFERENCE
   Pub/Sub → Cloud Run → Modelo → BigQuery
```

**Ejemplo: ETL Python en Cloud Run**

**Dockerfile:**
```dockerfile
FROM python:3.10-slim

WORKDIR /app

# Instalar dependencias
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copiar código
COPY main.py .

# Variable de entorno para puerto
ENV PORT=8080

# Comando de inicio
CMD exec gunicorn --bind :$PORT --workers 1 --threads 8 --timeout 0 main:app
```

**main.py:**
```python
from flask import Flask, request, jsonify
from google.cloud import bigquery, storage
import pandas as pd

app = Flask(__name__)
bq_client = bigquery.Client()
storage_client = storage.Client()

@app.route('/process-sales', methods=['POST'])
def process_sales():
    """
    Endpoint para procesar ventas desde GCS a BigQuery.
    Puede tomar >9 minutos sin problemas.
    """
    data = request.get_json()
    bucket_name = data.get('bucket')
    prefix = data.get('prefix', 'raw/sales/')
    
    # Leer todos los CSV desde GCS (proceso largo)
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)
    
    dfs = []
    for blob in blobs:
        if blob.name.endswith('.csv'):
            content = blob.download_as_bytes()
            df = pd.read_csv(pd.io.common.BytesIO(content))
            dfs.append(df)
    
    # Combinar y transformar
    df_combined = pd.concat(dfs, ignore_index=True)
    
    # Limpieza
    df_clean = df_combined.dropna()
    df_clean['total'] = df_clean['total'].astype(float)
    
    # Cargar a BigQuery (proceso largo)
    table_id = 'project.dataset.sales'
    job_config = bigquery.LoadJobConfig(
        write_disposition='WRITE_APPEND',
    )
    
    job = bq_client.load_table_from_dataframe(
        df_clean, 
        table_id, 
        job_config=job_config
    )
    job.result()  # Esperar a que termine
    
    return jsonify({
        'status': 'success',
        'rows_processed': len(df_clean),
        'table': table_id
    })

@app.route('/health', methods=['GET'])
def health():
    return 'OK', 200

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)
```

**requirements.txt:**
```
flask
gunicorn
google-cloud-bigquery
google-cloud-storage
pandas
```

**Deploy a Cloud Run:**
```bash
# Build container
gcloud builds submit --tag gcr.io/PROJECT_ID/sales-etl

# Deploy a Cloud Run
gcloud run deploy sales-etl \\
    --image gcr.io/PROJECT_ID/sales-etl \\
    --platform managed \\
    --region us-central1 \\
    --allow-unauthenticated \\
    --memory 2Gi \\
    --cpu 2 \\
    --timeout 3600 \\
    --max-instances 10 \\
    --set-env-vars PROJECT_ID=my-project

# Obtener URL
SERVICE_URL=$(gcloud run services describe sales-etl --region us-central1 --format 'value(status.url)')

# Invocar desde Cloud Scheduler (cron diario)
gcloud scheduler jobs create http daily-sales-etl \\
    --schedule="0 2 * * *" \\
    --uri="${SERVICE_URL}/process-sales" \\
    --http-method=POST \\
    --message-body='{"bucket":"my-bucket","prefix":"raw/sales/"}'
```

**Autoscaling y Concurrency:**

```bash
# Configurar autoscaling
gcloud run services update sales-etl \\
    --min-instances 0 \\
    --max-instances 100 \\
    --concurrency 10 \\
    --cpu-throttling \\
    --region us-central1

# 0 instancias cuando no hay tráfico (scale-to-zero)
# Hasta 100 instancias bajo carga
# 10 requests concurrentes por instancia
```

**VPC Connector (acceso a recursos privados):**

```bash
# Crear VPC connector
gcloud compute networks vpc-access connectors create my-connector \\
    --region us-central1 \\
    --range 10.8.0.0/28

# Configurar Cloud Run para usar VPC
gcloud run services update sales-etl \\
    --vpc-connector my-connector \\
    --region us-central1

# Ahora puede acceder a Cloud SQL, Memorystore (Redis), etc.
```

**Monitoring:**

```python
# Agregar logging estructurado
import logging
import json

logger = logging.getLogger(__name__)

# Log en formato JSON para Cloud Logging
log_entry = {
    'severity': 'INFO',
    'message': 'Processing batch',
    'rows': len(df),
    'duration_seconds': elapsed_time
}
print(json.dumps(log_entry))

# Métricas custom
from opencensus.ext.stackdriver import stats_exporter
exporter = stats_exporter.new_stats_exporter()
```

**Pricing:**
- CPU: $0.00002400 por vCPU-segundo
- Memory: $0.00000250 por GB-segundo
- Requests: $0.40 por 1M requests
- Networking: Egress estándar

**Ejemplo de costo:**
- 1,000 requests/día
- 2 vCPU, 2GB RAM
- 60s por request promedio
- **Costo mensual: ~$9**

**Ventajas de Cloud Run:**
- ✅ Sin límite de tiempo (vs 9 min Functions)
- ✅ Portabilidad (Docker estándar)
- ✅ Control total de dependencias
- ✅ gRPC nativo (ideal para ML serving)
- ✅ Blue/green deployments built-in
- ✅ Traffic splitting (canary releases)

---
**Autor:** Luis J. Raigoso V. (LJRV)

## 7. Comparación: GCP vs AWS vs Azure

### 🔄 **Multi-Cloud Comparison**

| Servicio | GCP | AWS | Azure |
|----------|-----|-----|-------|
| **Object Storage** | Cloud Storage | S3 | Blob Storage |
| **Data Warehouse** | BigQuery | Redshift | Synapse Analytics |
| **ETL Serverless** | Dataflow (Beam) | Glue (PySpark) | Data Factory |
| **Streaming** | Pub/Sub + Dataflow | Kinesis + Lambda | Event Hubs + Stream Analytics |
| **Orchestration** | Cloud Composer | MWAA (Airflow) | Data Factory |
| **Serverless Compute** | Cloud Functions | Lambda | Functions |
| **Notebooks** | Vertex AI Workbench | SageMaker | Machine Learning Studio |
| **ML Platform** | Vertex AI | SageMaker | Azure ML |

**Cuándo elegir cada cloud:**

**GCP:**
- ✅ BigQuery (mejor Data Warehouse serverless)
- ✅ Kubernetes-first (GKE es líder)
- ✅ ML/AI (TensorFlow, Vertex AI)
- ✅ Pricing transparente
- ❌ Menos servicios que AWS
- ❌ Menor presencia enterprise

**AWS:**
- ✅ Más servicios (200+)
- ✅ Mayor adoption (33% market share)
- ✅ Mejor documentación/comunidad
- ✅ Compliance certifications más amplio
- ❌ Pricing complejo
- ❌ Muchos servicios legacy

**Azure:**
- ✅ Integración Microsoft (Active Directory, Office 365)
- ✅ Híbrido (on-prem + cloud con Azure Arc)
- ✅ Windows workloads
- ❌ Curva de aprendizaje pronunciada
- ❌ Documentación inconsistente

**Arquitectura Multi-Cloud:**
```
On-Premise → Azure (ingesta + AD)
              ↓
          Cloud Storage (staging)
              ↓
       BigQuery (analytics) ← Looker (BI)
              ↓
      S3 (archival) → Glacier
```

**Portabilidad:**
- Terraform para IaC multi-cloud
- Apache Beam (portable Dataflow/Flink/Spark)
- Kubernetes para compute portable
- Parquet/Avro para data formats

---
**Autor:** Luis J. Raigoso V. (LJRV)

## 8. Ejercicios Prácticos

### 📝 **Ejercicios Hands-On**

**Ejercicio 1: Cloud Storage + BigQuery Pipeline**
- Objetivo: Crear un pipeline completo desde GCS a BigQuery
- Tareas:
  1. Crear bucket con lifecycle policy (Nearline después de 30 días)
  2. Subir archivo CSV particionado por fecha (`raw/ventas/2025/10/30/`)
  3. Crear dataset BigQuery
  4. Cargar CSV a tabla particionada por `DATE(fecha)`
  5. Ejecutar query agregada con costo < $0.01
  6. Validar con `INFORMATION_SCHEMA.JOBS_BY_PROJECT`

**Ejercicio 2: Dataflow ETL Batch**
- Objetivo: Procesar archivos CSV con Apache Beam
- Tareas:
  1. Crear script Beam que lea múltiples CSVs desde GCS
  2. Implementar transformaciones: filtrar nulos, validar rangos
  3. Agregar por `cliente_id` (total ventas, cantidad)
  4. Escribir resultados a BigQuery
  5. Ejecutar con DirectRunner (local) y DataflowRunner (cloud)
  6. Comparar tiempos y costos

**Ejercicio 3: Cloud Composer DAG Completo**
- Objetivo: Orquestar pipeline end-to-end
- Tareas:
  1. Task 1: `GCSObjectExistenceSensor` - validar archivos
  2. Task 2: `DataflowCreatePythonJobOperator` - ejecutar ETL
  3. Task 3: `BigQueryInsertJobOperator` - crear tabla agregada
  4. Task 4: `BigQueryCheckOperator` - validar calidad datos
  5. Task 5 (condicional): `EmailOperator` si hay errores
  6. Programar con `schedule_interval='@daily'`

**Ejercicio 4: Cloud Function Event-Driven**
- Objetivo: Procesar archivos automáticamente al subir
- Tareas:
  1. Crear function con trigger `google.storage.object.finalize`
  2. Validar schema CSV (columnas requeridas, tipos)
  3. Si válido → Publicar mensaje Pub/Sub topic `valid-data`
  4. Si inválido → Mover a `gs://bucket/errors/` y loguear
  5. Deploy y testear con `gsutil cp`
  6. Monitorear con Cloud Logging

**Ejercicio 5: Cloud Run ETL Largo**
- Objetivo: Procesar workload >9 minutos
- Tareas:
  1. Crear Dockerfile con Python + pandas + google-cloud-bigquery
  2. Implementar Flask app con endpoint `/process-sales`
  3. Leer 100+ archivos CSV desde GCS (simular carga pesada)
  4. Agregar y escribir a BigQuery
  5. Deploy a Cloud Run con 2GB RAM, timeout 3600s
  6. Configurar Cloud Scheduler para ejecución diaria 2 AM
  7. Implementar logging JSON estructurado

**Ejercicio 6: BigQuery Optimization Challenge**
- Objetivo: Reducir costos de queries
- Tareas:
  1. Crear tabla `ventas_raw` sin particionar (10M registros)
  2. Ejecutar query agregada y medir bytes escaneados
  3. Crear `ventas_partitioned` (por fecha)
  4. Crear `ventas_clustered` (partición + cluster por cliente)
  5. Comparar costos: raw vs partitioned vs clustered
  6. Implementar materialized view para agregaciones frecuentes
  7. Usar `APPROX_COUNT_DISTINCT` para estimaciones rápidas

**Ejercicio 7: BigQuery ML Predicción**
- Objetivo: Entrenar modelo sin salir de BigQuery
- Tareas:
  1. Crear modelo de clasificación: `churned` (0/1)
  2. Features: `total_compras`, `dias_ultima_compra`, `edad`
  3. Split train/test con `ML.SPLIT`
  4. Evaluar con `ML.EVALUATE`
  5. Predecir nuevos clientes con `ML.PREDICT`
  6. Exportar modelo para serving externo

**Recursos de Aprendizaje:**
- [GCP Free Tier](https://cloud.google.com/free) - $300 crédito por 90 días
- [BigQuery Sandbox](https://cloud.google.com/bigquery/docs/sandbox) - Sin billing requerido
- [Qwiklabs: Data Engineering](https://www.qwiklabs.com/quests/132)
- [Coursera: Data Engineering on GCP](https://www.coursera.org/specializations/gcp-data-machine-learning)
- [Cloud Run Quickstarts](https://cloud.google.com/run/docs/quickstarts)
- [Apache Beam Katas](https://github.com/apache/beam/tree/master/learning/katas)

**Tips para Practicar:**
- Usa datasets públicos de BigQuery para experimentar gratis
- Limita queries con `LIMIT` para reducir costos
- Prueba Dataflow con DirectRunner antes de cloud
- Usa `--dry-run` en gcloud para validar comandos
- Monitorea costos en [Cloud Billing](https://console.cloud.google.com/billing)

---
**Autor:** Luis J. Raigoso V. (LJRV)

## 9. Conclusión

### 🎯 **Key Takeaways de GCP para Data Engineering**

**Fortalezas de GCP:**

1. **BigQuery es excepcional:**
   - Queries extremadamente rápidas sobre TB/PB
   - Serverless (no tuning de clusters)
   - BigQuery ML integrado (modelos sin salir de SQL)
   - Flat-rate pricing predecible para cargas pesadas
   - Result caching automático (48h gratis)

2. **Cloud Run ofrece flexibilidad única:**
   - Sin límite de tiempo (vs 9 min Functions)
   - Contenedores Docker estándar (portabilidad)
   - Auto-scaling de 0 a miles de instancias
   - Ideal para ETL largo, APIs, ML serving

3. **Dataflow con Apache Beam:**
   - Unified model (mismo código batch + streaming)
   - Portable a otros runners (Flink, Spark)
   - Auto-scaling inteligente
   - Windowing avanzado para streaming

4. **Integración cohesiva:**
   - IAM unificado (service accounts)
   - Cloud Logging/Monitoring centralizado
   - Stackdriver para observabilidad
   - Vertex AI para ML lifecycle completo

**Limitaciones:**

- Menos servicios que AWS (pero más enfocados)
- Lock-in en BigQuery (aunque usa Standard SQL)
- Menor comunidad/recursos que AWS
- Algunas features enterprise menos maduras

**Comparación Final:**

| Aspecto | GCP | AWS | Azure |
|---------|-----|-----|-------|
| **DW Serverless** | ⭐⭐⭐⭐⭐ BigQuery | ⭐⭐⭐⭐ Athena | ⭐⭐⭐ Synapse Serverless |
| **Serverless Compute** | ⭐⭐⭐⭐⭐ Cloud Run | ⭐⭐⭐⭐ Lambda | ⭐⭐⭐ Functions |
| **Pricing Clarity** | ⭐⭐⭐⭐⭐ Muy claro | ⭐⭐⭐ Complejo | ⭐⭐⭐ Medio |
| **ML/AI** | ⭐⭐⭐⭐⭐ Vertex AI | ⭐⭐⭐⭐ SageMaker | ⭐⭐⭐ Azure ML |
| **Kubernetes** | ⭐⭐⭐⭐⭐ GKE (creadores) | ⭐⭐⭐⭐ EKS | ⭐⭐⭐⭐ AKS |
| **Enterprise** | ⭐⭐⭐ Menor adoption | ⭐⭐⭐⭐⭐ Líder | ⭐⭐⭐⭐ Microsoft stack |

**Cuándo elegir GCP:**

✅ **Elige GCP si:**
- BigQuery es tu DW principal (mejor del mercado)
- ML/AI es crítico (TensorFlow nativo, TPUs)
- Kubernetes-heavy (GKE es el mejor)
- Quieres pricing simple y transparente
- Startup/tech company (cultura cloud-native)

❌ **Evita GCP si:**
- Necesitas 200+ servicios especializados (AWS tiene más)
- Microsoft stack es mandatorio (Active Directory, etc.)
- Compliance muy específico (AWS tiene más certifications)
- Equipo ya experto en AWS/Azure

**Próximos Pasos:**

1. ✅ **Crear cuenta GCP** ([free tier $300](https://cloud.google.com/free))
2. ✅ **Completar Qwiklab:** [Data Engineering on GCP](https://www.qwiklabs.com/quests/132)
3. ✅ **Certificación:** [Professional Data Engineer](https://cloud.google.com/certification/data-engineer)
4. ✅ **Explorar:**
   - Vertex AI Pipelines (ML workflow)
   - Dataplex (data governance)
   - BigQuery Omni (multi-cloud)
   - Cloud Spanner (global SQL)

**Stack Recomendado GCP:**

```
Ingesta: Pub/Sub + Dataflow
Storage: Cloud Storage (raw) + BigQuery (curated)
Compute: Cloud Run (ETL) + Dataflow (batch/stream)
Orchestration: Cloud Composer (Airflow)
Serving: BigQuery + Looker (BI)
ML: Vertex AI + BigQuery ML
Governance: Dataplex + Data Catalog
```

**Happy data engineering en GCP! 🚀**

---
**Autor Final:** LuisRai (Luis J. Raigoso V.)  
© 2024-2025 - Data Engineering Modular Course

---

## 🧭 Navegación

**← Anterior:** [☁️ AWS para Ingeniería de Datos: S3, Glue, Athena y Lambda](03_cloud_aws.ipynb)

**Siguiente →:** [☁️ Azure para Ingeniería de Datos: ADLS, Synapse, Data Factory y Databricks →](03c_cloud_azure.ipynb)

**📚 Índice de Nivel Mid:**
- [⚡ Mid - 01. Orquestación de Pipelines con Apache Airflow](01_apache_airflow_fundamentos.ipynb)
- [Streaming con Apache Kafka: Fundamentos](02_streaming_kafka.ipynb)
- [☁️ AWS para Ingeniería de Datos: S3, Glue, Athena y Lambda](03_cloud_aws.ipynb)
- [☁️ GCP para Ingeniería de Datos: BigQuery, Cloud Storage, Dataflow y Composer](03b_cloud_gcp.ipynb) ← 🔵 Estás aquí
- [☁️ Azure para Ingeniería de Datos: ADLS, Synapse, Data Factory y Databricks](03c_cloud_azure.ipynb)
- [🗄️ Bases de Datos Relacionales y NoSQL: PostgreSQL y MongoDB](04_bases_datos_postgresql_mongodb.ipynb)
- [♻️ DataOps y CI/CD para Pipelines de Datos](05_dataops_cicd.ipynb)
- [🌐 Conectores Avanzados: REST, GraphQL y SFTP](06_conectores_avanzados_rest_graphql_sftp.ipynb)
- [🧩 Optimización SQL y Particionado de Datos](07_optimizacion_sql_particionado.ipynb)
- [🚀 Servicios de Datos con FastAPI](08_fastapi_servicios_datos.ipynb)
- [🧪 Proyecto Integrador Mid 1: API → DB → Parquet con Orquestación](09_proyecto_integrador_1.ipynb)
- [🔄 Proyecto Integrador Mid 2: Kafka → Streaming → Data Lake y Monitoreo](10_proyecto_integrador_2.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
